In [ ]:
# -*- coding: utf-8 -*-
import click
import logging
from pathlib import Path

from os import listdir
from os.path import isfile, join

import numpy as np
import soundfile as sf
from scipy import io
import scipy.signal as sp
from src.features import gtgram

import simpleaudio as sa


import matplotlib as mpl
import matplotlib.pyplot as plt
import src.features.filters as filters
import src.features.helpers_vis as hp_vis
import src.features.helpers as hp
import src.data.generateData as generate_data


ROOT = Path('.').resolve().parents[0]
# set the path to the sound files
SOUND_FILES = ROOT / 'data/raw/sound_samples/'
# create a list of the sound files
SOUND_FILES = list(SOUND_FILES.glob('**/*.wav'))


In [ ]:



def create_spectrum(freq_bands=24, snr=0.2, normalize=False, azimuth=12, time_window=0.1, max_freq=20000):

    
    str_r = 'data/processed_' + str(max_freq) + 'Hz/binaural_right_0_gammatone_' + str(time_window) + '_window_' + str(int(snr * 100)) + '_srn_' + str(freq_bands) + '_channels_' + str((azimuth - 12) * 10) + '_azi_' + str(normalize) + '_norm_no_hrtf.npy'
    str_l = 'data/processed_' + str(max_freq) + 'Hz/binaural_left_0_gammatone_' + str(time_window) + '_window_' + str(int(snr * 100)) + '_srn_' + str(freq_bands) + '_channels_' + str((azimuth - 12) * 10) + '_azi_' + str(normalize) + '_norm_no_hrtf.npy'

    path_data_r = ROOT / str_r
    path_data_l = ROOT / str_l
    
    print(path_data_r.as_posix())

    # check if we can load the data from a file
    if path_data_r.is_file() and path_data_l.is_file():
        print('Data set found. Loading from file : ' + str_r)
        print(path_data_l)
        return np.load(path_data_r), np.load(path_data_l)
    else:
        print('Creating data set : ' + str_l)
       
        # use always all elevations -> 50
        psd_all_i = np.zeros((len(SOUND_FILES), 25, freq_bands))
        psd_all_c = np.zeros((len(SOUND_FILES), 25, freq_bands))
        for i in range(0,psd_all_i.shape[0]):
            print("Creating dataset for sound: " + SOUND_FILES[i].name)
            for i_elevs in range(psd_all_i.shape[1]):
                # load a sound sample
                signal = sf.read(SOUND_FILES[i].as_posix())[0]

                # filter the signal
                signal_elevs = sp.filtfilt([1, 0], 1, signal)
                # add noise to the signal
                signal_elevs = (1 - snr) * signal_elevs + snr * np.random.random(signal_elevs.shape[0]) * signal.max()

                ##### Sound Playback #####
                # signal_play = signal_elevs * (2**15 - 1) / np.max(np.abs(signal_elevs))
                # signal_play = signal_play.astype(np.int16)
                #
                # # Start playback
                # play_obj = sa.play_buffer(signal_play, 1, 2, 44100)
                #
                # # Wait for playback to finish before exiting
                # play_obj.wait_done()

                # filter the signal
                signal_elevs_c = sp.filtfilt([1, 0], 1, signal)
                # add noise to the signal
                signal_elevs_c = (1 - snr) * signal_elevs_c + snr * np.random.random(signal_elevs_c.shape[0]) * signal.max()

                # Default gammatone-based spectrogram parameters
                time_window = 0.1
                twin = time_window
                thop = twin / 2
                fmin = 100
                fs = 44100

                ###### Apply Gammatone Filter Bank ##############
                # ipsi side
                y = gtgram.gtgram(signal_elevs, fs, twin,
                                  thop, freq_bands, fmin, max_freq)

                y = np.mean(y, axis=1)
                y = (20 * np.log10(y + np.finfo(np.float32).eps))
                psd_all_i[i, i_elevs, :] = y
                # contralateral side
                y = gtgram.gtgram(signal_elevs_c, fs,
                                  twin, thop, freq_bands, fmin, max_freq)
                y = np.mean(y, axis=1)
                y = (20 * np.log10(y + np.finfo(np.float32).eps))
                psd_all_c[i, i_elevs, :] = y
                #################################################

        np.save(path_data_r.absolute(), psd_all_c)
        np.save(path_data_l.absolute(), psd_all_i)

        return psd_all_c, psd_all_i



In [ ]:

########################################################################
######################## Set parameters ################################
########################################################################
normalize = False  # paramter is not considered

time_window = 0.1  # time window for spectrogram in sec

# Parameter to test
snr = 0.2  # Signal to noise ratio
freq_bands = 128 # Frequency bands in resulting data
azimuth = 12   # which azimuths to create
max_freq = 20000 # define max frequency for gammatone filter bank



save_figs=True
save_type='svg'
model_name='elevation_spectra_maps'
exp_name='figures_paper' 
elevations=25
clean=True
participant_number = 9

logger = logging.getLogger(__name__)
logger.info('Plotting elevation spectra map for different sounds')


elevations = np.arange(0, elevations, 1)

# make sure save type is given
if not save_type or len(save_type) == 0:
    save_type = 'svg'


exp_name_str = hp.create_exp_name([exp_name, time_window, int(snr * 100), freq_bands, max_freq,
                                   participant_number, (azimuth - 12) * 10, normalize, len(elevations)])
exp_path = ROOT / 'models' / model_name
exp_file = exp_path / exp_name_str


########################################################################
########################################################################

# create the spectrum data
spec_c, spec_i = create_spectrum(freq_bands, snr, normalize, azimuth, time_window, max_freq=max_freq)

# create the filtered HRTF data
psd_all_c, psd_all_i = generate_data.create_data(freq_bands, participant_number, snr, normalize, azimuth, time_window,  max_freq=max_freq)

vmin= -80
vmax= -40


fig_size = (7, 12)
# fig_size = (20, 14)

formatter = hp_vis.ERBFormatter(20, max_freq, unit='', places=0)


for i_sound, sound in enumerate(SOUND_FILES):
    sound = sound.name.split('.')[0]
    # IPSI
    fig = plt.figure(figsize=fig_size)
    ax = fig.add_subplot(2, 1, 1)
    ax.set_title(sound)
    data = np.squeeze(spec_i[i_sound])
    c = ax.pcolormesh(np.linspace(0, 1, data.shape[1]), np.linspace(-45, 90, data.shape[0]),
                      data, shading='gouraud', linewidth=0, rasterized=True,vmin=vmin, vmax=vmax)
    plt.colorbar(c)
    ax.xaxis.set_major_formatter(formatter)
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Elevations [deg]')
    # ax.set_yticklabels(t[1:-1])
    
    ax = fig.add_subplot(2, 1, 2)
    ax.set_title(sound)
    data = np.squeeze(psd_all_c[i_sound])
    c = ax.pcolormesh(np.linspace(0, 1, data.shape[1]), np.linspace(-45, 90, data.shape[0]),
                      data, shading='gouraud', linewidth=0, rasterized=True,vmin=vmin, vmax=vmax)
    plt.colorbar(c)
    ax.xaxis.set_major_formatter(formatter)
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Elevations [deg]')
    
    

    if save_figs:
        fig_save_path = ROOT / 'reports' / 'figures' / exp_name_str / model_name  / ('participant_' + str(participant_number))
        if not fig_save_path.exists():
            fig_save_path.mkdir(parents=True, exist_ok=True)
        path_final = (fig_save_path / (model_name + '_' + exp_name + '_raw_maps_ipsi_' + str(sound) + '.' + save_type)).as_posix()
        plt.savefig(path_final, dpi=300, transparent=True)
        print('Writing File :' + path_final)
        plt.close()
    else:
        plt.show()


